PHW#3S

Xinyun Wang

### **Proposition 1: Find All Salespeople Who Achieved Sales in 2014 but Did Not Achieve Sales in 2015**

#### **Functional Specification**

**Query Name:** Salespeople with Sales in 2014 but No Sales in 2015

**Description:** This query retrieves salespeople who made sales in 2014 but made no sales in 2015, using set operators to identify these salespeople.

**Inputs:**

- `BusinessEntityID`
- `OrderDate`

**Outputs:**

- `BusinessEntityID`
- `FirstName`
- `LastName`

**Steps:**

1. Select salespeople who made sales in 2014.
2. Select salespeople who made sales in 2015.
3. Use `EXCEPT` to find salespeople who achieved sales in 2014 but not in 2015.

#### **Assumptions**

- The data is available in the `Sales.SalesOrderHeader`, `Sales.SalesPerson`, `HumanResources.Employee`, and `Person.Person` tables.
- Each salesperson has a unique `BusinessEntityID`.
- `SalesPersonID` in `Sales.SalesOrderHeader` links to the salesperson.

#### **Example Output**

| BusinessEntityID | FirstName | LastName |
| --- | --- | --- |
| 274 | Rob | Walters |
| 275 | Gail | Erickson |

In [74]:
USE AdventureWorks2019

SELECT DISTINCT 
    SP.BusinessEntityID, P.FirstName, P.LastName
FROM 
    Sales.SalesPerson AS SP
JOIN 
    HumanResources.Employee AS E ON SP.BusinessEntityID = E.BusinessEntityID
JOIN 
    Person.Person AS P ON E.BusinessEntityID = P.BusinessEntityID
JOIN 
    Sales.SalesOrderHeader AS SOH ON SP.BusinessEntityID = SOH.SalesPersonID
WHERE 
    YEAR(SOH.OrderDate) = 2014
EXCEPT

SELECT DISTINCT 
    SP.BusinessEntityID, P.FirstName, P.LastName
FROM 
    Sales.SalesPerson AS SP
JOIN 
    HumanResources.Employee AS E ON SP.BusinessEntityID = E.BusinessEntityID
JOIN 
    Person.Person AS P ON E.BusinessEntityID = P.BusinessEntityID
JOIN 
    Sales.SalesOrderHeader AS SOH ON SP.BusinessEntityID = SOH.SalesPersonID
WHERE 
    YEAR(SOH.OrderDate) = 2015;



(17 rows affected)

Total execution time: 00:00:00.074

BusinessEntityID,FirstName,LastName
274,Stephen,Jiang
275,Michael,Blythe
276,Linda,Mitchell
277,Jillian,Carson
278,Garrett,Vargas
279,Tsvi,Reiter
280,Pamela,Ansman-Wolfe
281,Shu,Ito
282,José,Saraiva
283,David,Campbell


### **Proposition 2: Find Vendors Who Supply Components but Not Finished Goods**

#### **Functional Specification**

**Query Name:** Vendors Supplying Components but Not Finished Goods

**Description:** This query retrieves vendors who supply products in the **"Components"** category but do not supply products in the **"Finished Goods"** category. It uses the `EXCEPT` set operator to identify such vendors.

**Inputs:**

- `VendorID`
- `ProductCategory`

**Outputs:**

- `VendorID`
- `VendorName`

**Steps:**

1. Select vendors supplying products in the "Components" category.
2. Select vendors supplying products in the "Finished Goods" category.
3. Use `EXCEPT` to find vendors who supply "Components" but not "Finished Goods".

**Assumptions:**

- The data is available in the `Purchasing.Vendor`, `Purchasing.ProductVendor`, `Production.Product`, `Production.ProductSubcategory`, and `Production.ProductCategory` tables.
- Vendor names are retrieved from the `Purchasing.Vendor` table.

**Example Output**

| VendorID | VendorName |
| --- | --- |
| 1652 | Contoso Components |
| 1672 | Northwind Traders |

In [75]:
USE AdventureWorks2019

SELECT DISTINCT
    v.BusinessEntityID AS VendorID,
    v.Name AS VendorName
FROM
    Purchasing.Vendor AS v
INNER JOIN
    Purchasing.ProductVendor AS pv ON v.BusinessEntityID = pv.BusinessEntityID
INNER JOIN
    Production.Product AS p ON pv.ProductID = p.ProductID
INNER JOIN
    Production.ProductSubcategory AS ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
INNER JOIN
    Production.ProductCategory AS pc ON ps.ProductCategoryID = pc.ProductCategoryID
WHERE
    pc.Name = 'Components'

EXCEPT

SELECT DISTINCT
    v.BusinessEntityID AS VendorID,
    v.Name AS VendorName
FROM
    Purchasing.Vendor AS v
INNER JOIN
    Purchasing.ProductVendor AS pv ON v.BusinessEntityID = pv.BusinessEntityID
INNER JOIN
    Production.Product AS p ON pv.ProductID = p.ProductID
INNER JOIN
    Production.ProductSubcategory AS ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
INNER JOIN
    Production.ProductCategory AS pc ON ps.ProductCategoryID = pc.ProductCategoryID
WHERE
    pc.Name = 'Finished Goods';



(14 rows affected)

Total execution time: 00:00:00.018

VendorID,VendorName
1628,Bicycle Specialists
1638,Inline Accessories
1506,Greenwood Athletic Company
1570,First Rate Bicycles
1610,Hybrid Bicycle Center
1696,Chicago City Saddles
1586,Mitchell Sports
1508,"Compete Enterprises, Inc"
1658,Crowley Sport
1674,Varsity Sport Co.


### **Proposition 3: Find Employees Who Have Worked in Either the Marketing or Sales Departments but Not Both**

#### **Functional Specification**

**Query Name:** Employees in Marketing or Sales but Not Both

**Description:** This query retrieves employees who have been assigned to either the **"Marketing"** or **"Sales"** departments but not to both. It uses the `EXCEPT` and `UNION` set operators to identify such employees.

**Inputs:**

- `BusinessEntityID`
- `DepartmentName`

**Outputs:**

- `BusinessEntityID`
- `FirstName`
- `LastName`
- `DepartmentName`

**Steps:**

1. Select employees assigned to the "Marketing" department.
2. Select employees assigned to the "Sales" department.
3. Use `UNION` to combine both sets.
4. Use `EXCEPT` and `INTERSECT` to exclude employees in both departments.

**Assumptions:**

- The data is available in the `HumanResources.EmployeeDepartmentHistory`, `HumanResources.Department`, and `Person.Person` tables.
- Employee names are retrieved from the `Person.Person` table.

**Example Output**

| BusinessEntityID | FirstName | LastName | DepartmentName |
| --- | --- | --- | --- |
| 277 | Jillian | Carson | Sales |
| 278 | José | Saraiva | Marketing |

In [76]:
USE AdventureWorks2019

SELECT
    edh.BusinessEntityID,
    p.FirstName,
    p.LastName,
    d.Name AS DepartmentName
FROM
    HumanResources.EmployeeDepartmentHistory AS edh
INNER JOIN
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
INNER JOIN
    Person.Person AS p ON edh.BusinessEntityID = p.BusinessEntityID
WHERE
    d.Name = 'Marketing'

EXCEPT

SELECT
    edh.BusinessEntityID,
    p.FirstName,
    p.LastName,
    d.Name AS DepartmentName
FROM
    HumanResources.EmployeeDepartmentHistory AS edh
INNER JOIN
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
INNER JOIN
    Person.Person AS p ON edh.BusinessEntityID = p.BusinessEntityID
WHERE
    d.Name = 'Marketing'
INTERSECT
SELECT
    edh.BusinessEntityID,
    p.FirstName,
    p.LastName,
    d.Name AS DepartmentName
FROM
    HumanResources.EmployeeDepartmentHistory AS edh
INNER JOIN
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
INNER JOIN
    Person.Person AS p ON edh.BusinessEntityID = p.BusinessEntityID
WHERE
    d.Name = 'Sales'

UNION

SELECT
    edh.BusinessEntityID,
    p.FirstName,
    p.LastName,
    d.Name AS DepartmentName
FROM
    HumanResources.EmployeeDepartmentHistory AS edh
INNER JOIN
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
INNER JOIN
    Person.Person AS p ON edh.BusinessEntityID = p.BusinessEntityID
WHERE
    d.Name = 'Sales'

EXCEPT

SELECT
    edh.BusinessEntityID,
    p.FirstName,
    p.LastName,
    d.Name AS DepartmentName
FROM
    HumanResources.EmployeeDepartmentHistory AS edh
INNER JOIN
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
INNER JOIN
    Person.Person AS p ON edh.BusinessEntityID = p.BusinessEntityID
WHERE
    d.Name = 'Marketing'
INTERSECT
SELECT
    edh.BusinessEntityID,
    p.FirstName,
    p.LastName,
    d.Name AS DepartmentName
FROM
    HumanResources.EmployeeDepartmentHistory AS edh
INNER JOIN
    HumanResources.Department AS d ON edh.DepartmentID = d.DepartmentID
INNER JOIN
    Person.Person AS p ON edh.BusinessEntityID = p.BusinessEntityID
WHERE
    d.Name = 'Sales';


(28 rows affected)

Total execution time: 00:00:00.015

BusinessEntityID,FirstName,LastName,DepartmentName
16,David,Bradley,Marketing
17,Kevin,Brown,Marketing
18,John,Wood,Marketing
19,Mary,Dempsey,Marketing
20,Wanida,Benshoof,Marketing
21,Terry,Eminhizer,Marketing
22,Sariya,Harnpadoungsataya,Marketing
23,Mary,Gibson,Marketing
24,Jill,Williams,Marketing
250,Sheela,Word,Marketing


### **Proposition 4: Find Customers Who Made No Purchases in 2014**

#### **Functional Specification**

**Query Name:** Customers with No Purchases in 2014

**Description:** This query retrieves customers who did not make any purchases in the year 2014. It uses the `EXCEPT` set operator to identify customers without sales during that year.

**Inputs:**

- `CustomerID`
- `OrderDate`

**Outputs:**

- `CustomerID`
- `CustomerName`

**Steps:**

1. Select all customers.
2. Select customers who made purchases in 2014.
3. Use `EXCEPT` to find customers with no purchases in 2014.

**Assumptions:**

- The data is available in the `Sales.Customer`, `Sales.SalesOrderHeader`, `Person.Person`, and `Sales.Store` tables.
- Customer names are retrieved from `Person.Person` or `Sales.Store` tables.

**Example Output**

| CustomerID | CustomerName |
| --- | --- |
| 30015 | Bike Superstore |
| 30054 | Adventure Works |

In [77]:
USE AdventureWorks2019

SELECT DISTINCT
    c.CustomerID,
    ISNULL(s.Name, p.FirstName + ' ' + p.LastName) AS CustomerName
FROM
    Sales.Customer AS c
LEFT JOIN
    Person.Person AS p ON c.PersonID = p.BusinessEntityID
LEFT JOIN
    Sales.Store AS s ON c.StoreID = s.BusinessEntityID

EXCEPT

SELECT DISTINCT
    c.CustomerID,
    ISNULL(s.Name, p.FirstName + ' ' + p.LastName) AS CustomerName
FROM
    Sales.Customer AS c
INNER JOIN
    Sales.SalesOrderHeader AS soh ON c.CustomerID = soh.CustomerID
LEFT JOIN
    Person.Person AS p ON c.PersonID = p.BusinessEntityID
LEFT JOIN
    Sales.Store AS s ON c.StoreID = s.BusinessEntityID
WHERE
    YEAR(soh.OrderDate) = 2014;


(9466 rows affected)

Displaying Top 5000 rows.

Total execution time: 00:00:00.131

CustomerID,CustomerName
29484,Next-Door Bike Store
28866,Aaron Adams
29170,Alexandra Adams
18504,Angel Adams
13280,Bailey Adams
28678,Ben Adams
29491,Clamps & Brackets Co.
19822,Carlos Adams
26598,Charles Adams
19410,Chloe Adams


### **Proposition 5: Find Products That Have Never Been Purchased**

#### **Functional Specification**

**Query Name:** Products Never Purchased

**Description:** This query retrieves products that have never been sold. It uses the `EXCEPT` set operator to identify products without any sales records.

**Inputs:**

- `ProductID`

**Outputs:**

- `ProductID`
- `ProductName`

**Steps:**

1. Select all products from the `Production.Product` table.
2. Select products that have been sold from `Sales.SalesOrderDetail`.
3. Use `EXCEPT` to find products that have never been sold.

**Assumptions:**

- The data is available in the `Production.Product` and `Sales.SalesOrderDetail` tables.

**Example Output**

| ProductID | ProductName |
| --- | --- |
| 999 | New Product A |
| 1000 | New Product B |

In [78]:
USE AdventureWorks2019

SELECT
    p.ProductID,
    p.Name AS ProductName
FROM
    Production.Product AS p

EXCEPT

SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Sales.SalesOrderDetail AS sod
INNER JOIN
    Production.Product AS p ON sod.ProductID = p.ProductID;


(238 rows affected)

Total execution time: 00:00:00.016

ProductID,ProductName
1,Adjustable Race
3,BB Ball Bearing
2,Bearing Ball
316,Blade
324,Chain Stays
322,Chainring
320,Chainring Bolts
321,Chainring Nut
505,Cone-Shaped Race
323,Crown Race


### **Proposition 6: Find Products That Are Accessories or Clothing**

#### **Functional Specification**

**Query Name:** Products That Are Accessories or Clothing

**Description:** This query retrieves products that belong to either the **"Accessories"** or **"Clothing"** categories. It uses the `UNION` set operator to combine products from both categories.

**Inputs:**

- `ProductID`
- `ProductCategory`

**Outputs:**

- `ProductID`
- `ProductName`
- `CategoryName`

**Steps:**

1. Select products in the "Accessories" category.
2. Select products in the "Clothing" category.
3. Use `UNION` to combine both sets.

**Assumptions:**

- The data is available in the `Production.Product`, `Production.ProductSubcategory`, and `Production.ProductCategory` tables.

**Example Output**

| ProductID | ProductName | CategoryName |
| --- | --- | --- |
| 858 | Water Bottle - 30 oz. | Accessories |
| 937 | Long-Sleeve Logo Jersey | Clothing |

In [81]:
USE AdventureWorks2019

SELECT
    p.ProductID,
    p.Name AS ProductName,
    pc.Name AS CategoryName
FROM
    Production.Product AS p
INNER JOIN
    Production.ProductSubcategory AS ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
INNER JOIN
    Production.ProductCategory AS pc ON ps.ProductCategoryID = pc.ProductCategoryID
WHERE
    pc.Name = 'Accessories'

UNION

SELECT
    p.ProductID,
    p.Name AS ProductName,
    pc.Name AS CategoryName
FROM
    Production.Product AS p
INNER JOIN
    Production.ProductSubcategory AS ps ON p.ProductSubcategoryID = ps.ProductSubcategoryID
INNER JOIN
    Production.ProductCategory AS pc ON ps.ProductCategoryID = pc.ProductCategoryID
WHERE
    pc.Name = 'Clothing';


(64 rows affected)

Total execution time: 00:00:00.006

ProductID,ProductName,CategoryName
707,"Sport-100 Helmet, Red",Accessories
708,"Sport-100 Helmet, Black",Accessories
709,"Mountain Bike Socks, M",Clothing
710,"Mountain Bike Socks, L",Clothing
711,"Sport-100 Helmet, Blue",Accessories
712,AWC Logo Cap,Clothing
713,"Long-Sleeve Logo Jersey, S",Clothing
714,"Long-Sleeve Logo Jersey, M",Clothing
715,"Long-Sleeve Logo Jersey, L",Clothing
716,"Long-Sleeve Logo Jersey, XL",Clothing


### **Proposition 7: Find Products Sold Every Year from 2012 to 2014**

#### **Functional Specification**

**Query Name:** Products Sold Every Year from 2012 to 2014

**Description:** This query retrieves products that have been sold in every year from **2012** to **2014**. It uses the `INTERSECT` set operator to find products sold in all three years.

**Inputs:**

- `ProductID`
- `OrderDate`

**Outputs:**

- `ProductID`
- `ProductName`

**Steps:**

1. Select products sold in 2012.
2. Select products sold in 2013.
3. Select products sold in 2014.
4. Use `INTERSECT` to find products sold in all three years.

**Assumptions:**

- The data is available in the `Sales.SalesOrderHeader`, `Sales.SalesOrderDetail`, and `Production.Product` tables.

**Example Output**

| ProductID | ProductName |
| --- | --- |
| 680 | HL Road Frame - Black, 58 |
| 712 | Sport-100 Helmet, Red |

In [82]:
USE AdventureWorks2019

SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Sales.SalesOrderHeader AS soh
INNER JOIN
    Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN
    Production.Product AS p ON sod.ProductID = p.ProductID
WHERE
    YEAR(soh.OrderDate) = 2012

INTERSECT

SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Sales.SalesOrderHeader AS soh
INNER JOIN
    Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN
    Production.Product AS p ON sod.ProductID = p.ProductID
WHERE
    YEAR(soh.OrderDate) = 2013

INTERSECT

SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Sales.SalesOrderHeader AS soh
INNER JOIN
    Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN
    Production.Product AS p ON sod.ProductID = p.ProductID
WHERE
    YEAR(soh.OrderDate) = 2014;


(58 rows affected)

Total execution time: 00:00:00.146

ProductID,ProductName
856,"Men's Bib-Shorts, M"
779,"Mountain-200 Silver, 38"
762,"Road-650 Red, 44"
716,"Long-Sleeve Logo Jersey, XL"
862,"Full-Finger Gloves, M"
722,"LL Road Frame - Black, 58"
793,"Road-250 Black, 44"
822,"ML Road Frame-W - Yellow, 38"
799,"Road-550-W Yellow, 42"
836,"ML Road Frame-W - Yellow, 48"


### **Proposition 8: Find Products Sold in 'Northwest' Territory but Not in 'Southwest' Territory**

#### **Functional Specification**

**Query Name:** Products Sold in Northwest but Not Southwest Territory

**Description:** This query retrieves products that have been sold to customers in the **"Northwest"** sales territory but have not been sold to customers in the **"Southwest"** territory. It uses the `EXCEPT` set operator to identify these products.

**Inputs:**

- `ProductID`: The unique identifier for each product.
- `TerritoryID`: The unique identifier for each sales territory.
- `TerritoryName`: The name of the sales territory.

**Outputs:**

- `ProductID`: The unique identifier for the product.
- `ProductName`: The name of the product.

**Steps:**

1. Select products sold in the "Northwest" territory.
2. Select products sold in the "Southwest" territory.
3. Use `EXCEPT` to find products sold in "Northwest" but not in "Southwest".

**Assumptions:**

- Data is available in the `Sales.SalesOrderHeader`, `Sales.SalesOrderDetail`, `Production.Product`, and `Sales.SalesTerritory` tables.
- Products have unique `ProductID`s in the `Production.Product` table.

**Example Output**

| ProductID | ProductName |
| --- | --- |
| 680 | HL Road Frame - Black, 58 |
| 712 | Sport-100 Helmet, Red |

In [83]:
USE AdventureWorks2019

SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Sales.SalesOrderHeader AS soh
INNER JOIN
    Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN
    Production.Product AS p ON sod.ProductID = p.ProductID
INNER JOIN
    Sales.SalesTerritory AS st ON soh.TerritoryID = st.TerritoryID
WHERE
    st.Name = 'Northwest'

EXCEPT


SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Sales.SalesOrderHeader AS soh
INNER JOIN
    Sales.SalesOrderDetail AS sod ON soh.SalesOrderID = sod.SalesOrderID
INNER JOIN
    Production.Product AS p ON sod.ProductID = p.ProductID
INNER JOIN
    Sales.SalesTerritory AS st ON soh.TerritoryID = st.TerritoryID
WHERE
    st.Name = 'Southwest';


(2 rows affected)

Total execution time: 00:00:00.093

ProductID,ProductName
744,"HL Mountain Frame - Black, 44"
898,"LL Touring Frame - Blue, 62"


### **Proposition 9: Find Vendors Who Supply Products Not Sold in 2014**

#### **Functional Specification**

**Query Name:** Vendors Supplying Products Not Sold in 2014

**Description:** This query retrieves vendors who supply products that were not sold at all in the year 2014. It uses the `EXCEPT` set operator to find vendors whose products did not have any sales in that year.

**Inputs:**

- `VendorID`
- `ProductID`
- `OrderDate`

**Outputs:**

- `VendorID`
- `VendorName`

**Steps:**

1. Select vendors and products they supply.
2. Select products that were sold in 2014.
3. Use `EXCEPT` to find vendors whose products were not sold in 2014.

**Assumptions:**

- Data is available in the `Purchasing.Vendor`, `Purchasing.ProductVendor`, `Production.Product`, and `Sales.SalesOrderDetail` tables.
- Vendor names are retrieved from the `Purchasing.Vendor` table.

**Example Output**

| VendorID | VendorName |
| --- | --- |
| 1510 | International |
| 1516 | Gardner Touring Cycles |

In [84]:
USE AdventureWorks2019;

WITH VendorProducts AS (
    SELECT DISTINCT
        v.BusinessEntityID AS VendorID,
        v.Name AS VendorName,
        p.ProductID
    FROM
        Purchasing.Vendor AS v
    INNER JOIN
        Purchasing.ProductVendor AS pv ON v.BusinessEntityID = pv.BusinessEntityID
    INNER JOIN
        Production.Product AS p ON pv.ProductID = p.ProductID
)


, ProductsSoldIn2019 AS (
    SELECT DISTINCT
        sod.ProductID
    FROM
        Sales.SalesOrderDetail AS sod
    INNER JOIN
        Sales.SalesOrderHeader AS soh ON sod.SalesOrderID = soh.SalesOrderID
    WHERE
        YEAR(soh.OrderDate) = 2014
)

SELECT DISTINCT
    vp.VendorID,
    vp.VendorName
FROM
    VendorProducts AS vp

EXCEPT

SELECT DISTINCT
    vp.VendorID,
    vp.VendorName
FROM
    VendorProducts AS vp
INNER JOIN
    ProductsSoldIn2019 AS psi ON vp.ProductID = psi.ProductID;


(61 rows affected)

Total execution time: 00:00:00.074

VendorID,VendorName
1492,Australia Bike Retailer
1494,Allenson Cycles
1496,Advanced Bicycles
1500,Morgan Bike Accessories
1504,Chicago Rent-All
1510,International
1514,Training Systems
1516,Gardner Touring Cycles
1518,International Trek Center
1522,First National Sport Co.


### **Proposition 10: Find Products Reviewed in 2013 but Not in 2014**

#### **Functional Specification**

**Query Name:** Products Reviewed in 2013 but Not in 2014

**Description:** This query retrieves products that received customer reviews in the year 2013 but did not receive any reviews in 2014. It uses the `EXCEPT` set operator to identify these products.

**Inputs:**

- `ProductID`: The unique identifier for each product.
- `ReviewDate`: The date when the review was written.

**Outputs:**

- `ProductID`: The unique identifier for the product.
- `ProductName`: The name of the product.

**Steps:**

1. Select distinct products that have reviews dated in 2013.
2. Select distinct products that have reviews dated in 2014.
3. Use `EXCEPT` to find products reviewed in 2013 but not in 2014.

**Assumptions:**

- The data is available in the `Production.Product` and `Production.ProductReview` tables.
- Products have unique `ProductID`s in the `Production.Product` table.

**Example Output**

| ProductID | ProductName |
| --- | --- |
| 680 | HL Road Frame - Black, 58 |
| 712 | Sport-100 Helmet, Red |

In [85]:
USE AdventureWorks2019
SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Production.Product AS p
INNER JOIN
    Production.ProductReview AS pr ON p.ProductID = pr.ProductID
WHERE
    YEAR(pr.ReviewDate) = 2013

EXCEPT

SELECT DISTINCT
    p.ProductID,
    p.Name AS ProductName
FROM
    Production.Product AS p
INNER JOIN
    Production.ProductReview AS pr ON p.ProductID = pr.ProductID
WHERE
    YEAR(pr.ReviewDate) = 2014;


(3 rows affected)

Total execution time: 00:00:00.003

ProductID,ProductName
709,"Mountain Bike Socks, M"
798,"Road-550-W Yellow, 40"
937,HL Mountain Pedal
